### Часть 1: импорт библиотек и текст

Для дальнейшей обработки текста сразу скачаем и импортнем данные трех библиотек для Python: PyMystem3, PyMorphy2 и Natasha

In [ ]:
!pip install pymystem3==0.1.10

In [ ]:
!pip install pymorphy2

In [ ]:
!pip install natasha

In [4]:
import pandas as pd
import re
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem

In [10]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

Теперь о самом тексте: он был составлен из отрывков различных грамматических справок о субстантивации — когда другие части речи переходят в класс существительных. Большую часть текста составляют именно примеры с субстантивами (иногда — с их исходными прилагательными) из маленьких статей и НКРЯ. Поскольку текст изначально нужно было разметить самостоятельно, то при составлении текста я убирала пунктуацию и приводила все к нижнему регистру. Кроме того, сразу размечала слова в Excel.

Обозначения POS: ADVB-наречие, ADJF-прилагательное, CONJ-союз, DET-детерминатив, NOUN-существительное, NUMB-числительное, PREP-предлог, PRON-Местоимение, PRTF-полное причастие, PRTS-краткое причастие, PTCL-частица, V-глагол

In [5]:
xl = pd.ExcelFile(r"substext.xlsx")
df_1 = xl.parse("Лист1")
df_1

,word,POS
0,иногда,ADVB
1,я,PRON
2,хочу,V
3,подумать,V
4,о,PREP
...,...,...
214,и,CONJ
215,содержательным,ADJF
216,воспоминанием,NOUN
217,о,PREP


Текст сам по себе нам тоже понадобится, поэтому сохраним его в переменной *text*:

In [6]:
text = ' '.join(df_1["word"])
text

'иногда я хочу подумать о различных сложных и неоднозначных моментах в русском в плане определения части речи по большей части буду говорить про субстантивацию это явление можно считать древним но в то же время развивающимся процессом например некоторые прилагательные очень давно перешли в существительные к ним относятся существительные обозначающие фамилии и названия населённых пунктов иванов петров никитин фомин марьино митино существительное полученное в результате субстантивации называется субстантиватом исходное прилагательное мотивирующим прилагательным полная субстантивация когда прилагательное полностью перешло в существительное как прилагательное оно уже не может использоваться приведем примеры где такие субстантивы употреблены в предложениях я только возил каталки с больными в перевязочную на рентген в операционную его тугое ухо уловило со стороны ванной странный звук я буду ходить с сашей к портным этот город полон любовью там можно даже камню на мостовой сказать люблю прида

### Часть 2: POS Tagging

PyMorphy2:

In [7]:
%%time
morph = MorphAnalyzer() 
pyms_info = [morph.parse(word) for word in text.split()]
pyms = [str(word_tag[0].tag)[:4] for word_tag in pyms_info]  

CPU times: user 179 ms, sys: 33 ms, total: 212 ms
Wall time: 222 ms


PyMystem3:

In [8]:
%%time
mstm = Mystem()
mstms_info = mstm.analyze(text)
del(mstms_info[-1])
mstms = []
for word in mstms_info:
    if word['text'] != ' ':
        tag_line = str(word['analysis'][0]['gr'])
        pos = re.findall(r'[A-Z]*', tag_line)
        mstms.append(pos[0])

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz


CPU times: user 346 ms, sys: 86.2 ms, total: 433 ms
Wall time: 4.23 s


Natasha:

In [11]:
%%time
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

doc = Doc(text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

nats = []
for line in doc.sents[0].morph:
    for word in line:
        nats.append(word.pos)

CPU times: user 745 ms, sys: 347 ms, total: 1.09 s
Wall time: 951 ms


Добавим результаты в таблицу:

In [12]:
df_1["Pymorphy POS"] = pyms
df_1["Mystem POS"] = mstms
df_1["Natasha POS"] = nats

### Часть 3: Единый стандарт и Accuracy

In [13]:
def one_standard(some_df):
    some_df = some_df.replace('ADJ', 'ADJF')
    some_df = some_df.replace('A', 'ADJF')
    some_df = some_df.replace('APRO', 'DET')
    some_df = some_df.replace('ADV', 'ADVB')
    some_df = some_df.replace('ADVPRO', 'ADVB')
    some_df = some_df.replace('S', 'NOUN')
    some_df = some_df.replace('NUM', 'NUMB')
    some_df = some_df.replace('CCONJ', 'CONJ')
    some_df = some_df.replace('SCONJ', 'CONJ')
    some_df = some_df.replace('PR', 'PREP')
    some_df = some_df.replace('ADP', 'PREP')
    some_df = some_df.replace('NPRO', 'PRON')
    some_df = some_df.replace('SPRO', 'PRON')
    some_df = some_df.replace('PTCL', 'PART')
    some_df = some_df.replace('PRCL', 'PART')
    some_df = some_df.replace('V', 'VERB')
    some_df = some_df.replace('PRED', 'VERB')
    some_df = some_df.replace('INFN', 'VERB')
    some_df = some_df.replace('AUX', 'VERB')
    return some_df

In [14]:
df_1 = one_standard(df_1)

In [15]:
df_1

,word,POS,Pymorphy POS,Mystem POS,Natasha POS
0,иногда,ADVB,ADVB,ADVB,ADVB
1,я,PRON,PRON,PRON,PRON
2,хочу,VERB,VERB,VERB,VERB
3,подумать,VERB,VERB,VERB,VERB
4,о,PREP,PREP,PREP,PREP
...,...,...,...,...,...
214,и,CONJ,CONJ,CONJ,CONJ
215,содержательным,ADJF,ADJF,ADJF,ADJF
216,воспоминанием,NOUN,NOUN,NOUN,NOUN
217,о,PREP,PREP,PREP,PREP


In [16]:
from sklearn.metrics import accuracy_score

In [17]:
pym_accr = accuracy_score(df_1["POS"], df_1["Pymorphy POS"])
mstm_accr = accuracy_score(df_1["POS"], df_1["Mystem POS"])
nat_accr = accuracy_score(df_1["POS"], df_1["Natasha POS"])

print(f"Accuracy:\npymorphy: {pym_accr}\npymystem: {mstm_accr}\nNatasha: {nat_accr}")

Accuracy:
pymorphy: 0.863013698630137
pymystem: 0.8767123287671232
Natasha: 0.8493150684931506


Вывод: MyStem выполнил задачу лучше всего

### Часть 4

In [18]:
df_2 = df_1[["word", "Mystem POS"]]
df_2

,word,Mystem POS
0,иногда,ADVB
1,я,PRON
2,хочу,VERB
3,подумать,VERB
4,о,PREP
...,...,...
214,и,CONJ
215,содержательным,ADJF
216,воспоминанием,NOUN
217,о,PREP


Попробуем найти н-граммы трех типов: прил + сущ, предлог + сущ, глагол + сущ (объект)

In [19]:
def two_gram(df, POS1, POS2):
    ngrams = []
    for i, pos in enumerate(df["Mystem POS"]):
        if df["Mystem POS"][i] == POS1:
            if df["Mystem POS"][i+1] == POS2:
                match = df["word"][i], df["word"][i+1]
                ngrams.append(match)
    return ngrams

In [20]:
adjf_nouns = two_gram(df_2, "ADJF", "NOUN")
prep_nouns = two_gram(df_2, "PREP", "NOUN")
verb_nouns = two_gram(df_2, "VERB", "NOUN")

Первые два типа н-грамм вышли хорошо, есть пары даже с субстантивами:

In [21]:
adjf_nouns

[('неоднозначных', 'моментах'),
 ('большей', 'части'),
 ('населённых', 'пунктов'),
 ('исходное', 'прилагательное'),
 ('полная', 'субстантивация'),
 ('тугое', 'ухо'),
 ('странный', 'звук'),
 ('полон', 'любовью'),
 ('неполная', 'субстантивация'),
 ('русский', 'народ'),
 ('военный', 'врач'),
 ('жаркое', 'лето'),
 ('горячее', 'солнце'),
 ('больные', 'пациенты'),
 ('пожарной', 'команде'),
 ('хорошим', 'пожарным'),
 ('содержательным', 'воспоминанием')]

In [22]:
prep_nouns

[('в', 'плане'),
 ('про', 'субстантивацию'),
 ('в', 'результате'),
 ('в', 'существительное'),
 ('в', 'предложениях'),
 ('с', 'больными'),
 ('в', 'перевязочную'),
 ('на', 'рентген'),
 ('со', 'стороны'),
 ('с', 'сашей'),
 ('к', 'портным'),
 ('на', 'мостовой'),
 ('по', 'договору'),
 ('на', 'диване'),
 ('перед', 'телевизором'),
 ('с', 'мороженым'),
 ('о', 'прошлом')]

Третий тип (глагол с дополнением) получился не очень качественным. Так, не были распознаны причастия, есть пример и с неугаданным субстантивом. Впрочем, такую ситуацию можно было бы подкорректировать, используя больше данных из морф.анализа

In [23]:
verb_nouns

[('развивающимся', 'процессом'),
 ('обозначающие', 'фамилии'),
 ('называется', 'субстантиватом'),
 ('мотивирующим', 'прилагательным'),
 ('приведем', 'примеры'),
 ('возил', 'каталки'),
 ('люблю', 'приданое'),
 ('выздоравливающие', 'люди'),
 ('выздоравливающие', 'папа')]

### Часть 5

Шаблон 1: "далеко" + "не" + ADJF|DET|ADV

1) *Фильм далеко не такой отличный, как о нем говорят.*

2) *...сюжет далеко не нов...*

Шаблон 2: "далек" + "от" 

3) *получился не просто далёким от шедевра, но и..*

Словосочетания такого типа можно иногда встретить именно в отрицательных отзывых, когда пользователи как бы возражают положительному, более распространенному мнению. 

Шаблон 3, а точнее просто идея: Имена и все, что может быть PROPN в пост-тэггинге от Наташи

Так, если у нас в словаре будет "Томми Ли Джонс", а не 3 токена по отдельности, то, во-первых, мы потенциально можем чуть-чуть сэкономить место в словаре. Во-вторых, вероятность вычислить положительный отзыв, имея в словаре "Томми Ли Джонс", будет выше, чем в случае с "Томми", т.к. например речь может идти о Томми Вайсо (no offence). 